# Generate daily VWAP using dedicated service with or without corporate actions adjustments - Python

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot **on-demand** VWAP price from a dedicated `daily data store`.

### Inputs/outputs
Daily VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the daily VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows the differences when adjusting the bars with corporate actions.

### Services used
This sample uses gRPC requests in order to retrieve VWAP indicator from the hosted service. The queried endpoint in this script are:
* *DailyVwapsService*: to directly retrieve daily VWAP objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly

In [ ]:
pip install systemathics.apis==2.32.*

In [ ]:
import os
import csv
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_vwap_pb2 as daily_vwap
import systemathics.apis.services.daily.v1.daily_vwap_pb2_grpc as daily_vwap_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'ACA'
exchange = 'XPAR'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# Non adjusted
request = daily_vwap.DailyVwapsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = False
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily prices service
        service = daily_vwap_service.DailyVwapsServiceStub(channel)
        
        # process the daily prices request
        response = service.DailyVwaps(
            request = request, 
            metadata = [('authorization', token)]
        )
        print("Total daily prices retrieved: ",len(response.data))
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# With adjustments
request_adjusted = daily_vwap.DailyVwapsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = True
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily prices service
        service = daily_vwap_service.DailyVwapsServiceStub(channel)
        
        # process the daily prices request
        response_adjusted = service.DailyVwaps(
            request = request_adjusted, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
print("Total vwaps retrieved without adjustments: ", len(response.data))
print("Total vwaps retrieved with adjustments: ", len(response_adjusted.data))

### Step 4: Export data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
#pPrepare the data frame content
dates=[datetime(v.date.year, v.date.month, v.date.day) for v in response.data]
vwaps = [v.price for v in response.data]
volumes = [v.volume for v in response.data]
scores = [v.score for v in response.data]

vwaps_adjusted = [v.price for v in response_adjusted.data]
volumes_adjusted = [v.volume for v in response_adjusted.data]
scores_adjusted = [v.score for v in response_adjusted.data]

d = {'Date': dates, 'Vwap': vwaps,'Volume': volumes,'Score': scores, 'Vwap_adjusted': vwaps_adjusted, 'Volume_adjusted':volumes_adjusted,'Score_adjusted': scores_adjusted}

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)
df

In [ ]:
os.makedirs('output', exist_ok=True)
filename = 'output/{0}-{1}-dailyvwaps-comparison.csv'.format(ticker, exchange)
df.to_csv(filename, index=False)

### 5 Plot prices in both cases
The following code snippets display daily bars' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

#### 5.1 Plot non adjusted daily prices

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Set date as index
#df = df.set_index('Date')
#df

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Vwap'], name='Vwap'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name="Size"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Daily prices and sizes for {0}-{1} without corporate actions adjustments'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()

#### 5.2 Plot daily prices adjusted with corporate actions

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['Date'], y=df['Vwap_adjusted'], name='Vwap_adjusted'), secondary_y=True)
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume_adjusted'], name="Size_adjusted"), secondary_y=False)

fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Daily Vwap and sizes for {0}-{1} with corporate actions adjustments'.format(ticker,exchange))
fig.layout.yaxis2.showgrid=False
fig.show()